In [ ]:
from pathlib import Path
import datetime
import hmac
import hashlib
import json
import time
import threading
import queue
import asyncio

import numpy as np
import websocket
import polars as pl
import requests
from pydantic import BaseModel

import stock

In [ ]:
class APICert(BaseModel):
    api_key: str
    api_secret: str

In [ ]:
cert_file = stock.PROJECT_ROOT / "cert/gmo_api.json"
with open(cert_file, "r") as f:
    cert = APICert.model_validate_json(f.read())

In [ ]:
cert.api_key

In [ ]:
apiKey    = cert.api_key
secretKey = cert.api_secret

timestamp = '{0}000'.format(int(time.mktime(datetime.now().timetuple())))
method    = 'GET'
endPoint  = 'https://api.coin.z.com/private'
path      = '/v1/account/margin'

text = timestamp + method + path
sign = hmac.new(bytes(secretKey.encode('ascii')), bytes(text.encode('ascii')), hashlib.sha256).hexdigest()

headers = {
    "API-KEY": apiKey,
    "API-TIMESTAMP": timestamp,
    "API-SIGN": sign
}

res = requests.get(endPoint + path, headers=headers)
print (json.dumps(res.json(), indent=2))

In [ ]:
endPoint = 'https://api.coin.z.com/public'
path     = '/v1/klines?symbol=BTC&interval=1min&date=20210417'

response = requests.get(endPoint + path)
data = response.json()["data"]